In [12]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [13]:
# check version number
import imblearn
print(imblearn.__version__)

0.7.0


In [14]:
from sklearn.datasets import fetch_openml

df, y = fetch_openml('adult', version=2, as_frame=True, return_X_y=True)
# we are dropping the following features:
# - "fnlwgt": this feature was created while studying the "adult" dataset.
#   Thus, we will not use this feature which is not acquired during the survey.
# - "education-num": it is encoding the same information than "education".
#   Thus, we are removing one of these 2 features.
df = df.drop(columns=['fnlwgt', 'education-num'])

In [15]:
df.head()

age  workclass     education      marital-status         occupation  \
0  25.0    Private          11th       Never-married  Machine-op-inspct   
1  38.0    Private       HS-grad  Married-civ-spouse    Farming-fishing   
2  28.0  Local-gov    Assoc-acdm  Married-civ-spouse    Protective-serv   
3  44.0    Private  Some-college  Married-civ-spouse  Machine-op-inspct   
4  18.0        NaN  Some-college       Never-married                NaN   

  relationship   race     sex  capital-gain  capital-loss  hours-per-week  \
0    Own-child  Black    Male           0.0           0.0            40.0   
1      Husband  White    Male           0.0           0.0            50.0   
2      Husband  White    Male           0.0           0.0            40.0   
3      Husband  Black    Male        7688.0           0.0            40.0   
4    Own-child  White  Female           0.0           0.0            30.0   

  native-country  
0  United-States  
1  United-States  
2  United-States  
3  United-States  
4  United-States

In [16]:
classes_count = y.value_counts()
classes_count

<=50K    37155
>50K     11687
Name: class, dtype: int64

In [17]:
from imblearn.datasets import make_imbalance

ratio = 30
df_res, y_res = make_imbalance(
    df, y, sampling_strategy={
        classes_count.idxmin(): classes_count.max() // ratio
    }
)
y_res.value_counts()

<=50K    37155
>50K      1238
Name: class, dtype: int64

In [18]:
df_res.isnull().sum()

age                  0
workclass         2570
education            0
marital-status       0
occupation        2580
relationship         0
race                 0
sex                  0
capital-gain         0
capital-loss         0
hours-per-week       0
native-country     658
dtype: int64

In [19]:
X = df_res
y= y_res.values

In [20]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

print("X.shape: {} y.shape: {}".format(X.shape, y.shape))

X.shape: (38393, 12) y.shape: (38393,)


Podzielmy zbiór na train/test

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [22]:
print(np.unique(y_train, return_counts=True))
print(np.unique(y_test, return_counts=True))

(array([0, 1]), array([29725,   989], dtype=int64))
(array([0, 1]), array([7430,  249], dtype=int64))


# Zad

Robimy StratifiedKFold i znajdujemy optymalne parametry dla



* MultinomialNB (bez redukcji wymiarowości)
* LogisticRegression
* LinearSVC
* SVC
* KNeighborsClassifier
* DecisionTreeClassifier
* RandomForestClassifier
* BaggingClassifier
* ExtraTreesClassifier
* AdaBoostClassifier
* GradientBoostingClassifier
* VotingClassifier
* xgboost.XGBClassifier

## Dane są niezbalansowane, wykorzystaj 

```python
from imblearn.over_sampling import SMOTE, ADASYN
```

In [23]:
from sklearn.base import BaseEstimator, TransformerMixin

# A class to select numerical or categorical columns 
# since Scikit-Learn doesn't handle DataFrames yet
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names]

In [24]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer



num_pipeline = Pipeline([
        ("select_numeric", DataFrameSelector(["hours-per-week"])),
        ("imputer", SimpleImputer(strategy="median")),
    ])

In [25]:
# Inspired from stackoverflow.com/questions/25239958
class MostFrequentImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.most_frequent_ = pd.Series([X[c].value_counts().index[0] for c in X],
                                        index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.most_frequent_)

In [26]:
# from future_encoders import OneHotEncoder
from sklearn.preprocessing import OneHotEncoder

cat_pipeline = Pipeline([
        ("select_cat", DataFrameSelector(["workclass", "education", "occupation"])),
        ("imputer", MostFrequentImputer()),
        ("cat_encoder", OneHotEncoder(sparse=False, handle_unknown = 'ignore')),
    ])

In [27]:
from sklearn.pipeline import FeatureUnion
preprocess_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("cat_pipeline", cat_pipeline),
    ])

In [28]:
from sklearn.model_selection import StratifiedKFold

seed=123
kfold = StratifiedKFold(n_splits=5, random_state=seed,shuffle=True)

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
# from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

pipe = Pipeline([
    ('preprocessing', preprocess_pipeline), 
    ('scaler', StandardScaler()),
    ('over', SMOTE()),
    ('classifier', LogisticRegression(C=1, solver='newton-cg'))])


param_grid = {
            'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100]
}

grid_1 = GridSearchCV(pipe, param_grid, cv=kfold)

grid_1.fit(X_train, y_train)
grid_1.best_params_

{'classifier__C': 0.001}

In [30]:
from sklearn import  metrics


models = []
models.append(('LR', grid_1.best_estimator_))



precision_score = []
recall_score = []
f1_score = []
accuracy_score = []
b_accuracy_score = []
for name, model in models:
    print(name)
    print("precision_score: {}".format(metrics.precision_score(y_test, model.predict(X_test)) ))
    print("recall_score: {}".format( metrics.recall_score(y_test, model.predict(X_test)) ))
    print("f1_score: {}".format( metrics.f1_score(y_test, model.predict(X_test)) ))
    print("accuracy_score: {}".format( metrics.accuracy_score(y_test, model.predict(X_test)) ))
    precision_score.append(metrics.precision_score(y_test, model.predict(X_test)))
    recall_score.append(metrics.recall_score(y_test, model.predict(X_test)))
    f1_score.append( metrics.f1_score(y_test, model.predict(X_test)))
    accuracy_score.append(metrics.accuracy_score(y_test, model.predict(X_test)))
    
    
    b_accuracy_score.append(metrics.balanced_accuracy_score(y_test, model.predict(X_test)))

LR
precision_score: 0.08100558659217877
recall_score: 0.6987951807228916
f1_score: 0.14518147684605756
accuracy_score: 0.7331683812996483


In [31]:
import pandas as pd
d = {'precision_score': precision_score, 
     'recall_score': recall_score, 
     'f1_score': f1_score,
     'accuracy_score' : accuracy_score,
     'balanced_accuracy_score' : b_accuracy_score
    }
df = pd.DataFrame(data=d)
df.insert(loc=0, column='Method', value=['LR'])
df

Method  precision_score  recall_score  f1_score  accuracy_score  \
0     LR         0.081006      0.698795  0.145181        0.733168   

   balanced_accuracy_score  
0                 0.716558